# Hackanoho 2025

## Loading libraries and getting census data

In [23]:
# load some libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import zipfile
import urllib.request
import ssl
import shutil

In [13]:
# look at the files in the data directory; create it if it doesn't exist
DATA_DIR = 'data'
os.makedirs(DATA_DIR, exist_ok=True)
os.listdir(DATA_DIR)

['Statistical Area 1 dataset for Census 2018 - total New Zealand - CSV_updated_16-7-20',
 'Statistical Area 1 dataset for Census 2018 - total New Zealand - CSV_updated_16-7-20.zip']

In [26]:
# Create a context that doesn't verify certificates
ssl_context = ssl._create_unverified_context()

# download the SA1-level 2018 census data from Stats NZ
ZIP_PATH = os.path.join(DATA_DIR, '2018-census.zip')
URL =   'https://www3.stats.govt.nz/2018census/SA1Dataset/Statistical%20Area%201%20dataset%20for%20Census%202018%20-%20total%20New%20Zealand%20-%20CSV_updated_16-7-20.zip'

# download the file if it doesn't exist
if not os.path.exists(ZIP_PATH):
    print('Downloading', URL)
    # Create a request with a browser-like user agent
    req = urllib.request.Request(
        URL, 
        data=None, 
        headers={
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'
        }
    )
    
    # Use urlopen with context parameter and user agent
    with urllib.request.urlopen(req, context=ssl_context) as response, open(ZIP_PATH, 'wb') as out_file:
        shutil.copyfileobj(response, out_file)
    
    with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
        print('Extracting', ZIP_PATH)
        zip_ref.extractall(DATA_DIR)
else:
    print(f"File already exists at {ZIP_PATH}")

# look at the files in the data directory
os.listdir(DATA_DIR)

File already exists at data/2018-census.zip


['.DS_Store',
 '2018-census.zip',
 'Individual_part3a_totalNZ-wide_format_updated_16-7-20.csv',
 'Individual_part3b_totalNZ-wide_format_updated_16-7-20.csv',
 'Dwellings_totalNZ-wide_format_updated_16-7-20.csv',
 'Individual_part2_totalNZ-wide_format_updated_16-7-20.csv',
 'Households_totalNZ-wide_format_updated_16-7-20.csv',
 'Individual_part1_totalNZ-wide_format_updated_16-7-20.csv']

In [27]:
# load the csv in DATA_DIR starting with 'Individual_part1'
# and print the first few rows
file = [f for f in os.listdir(DATA_DIR) if f.startswith('Individual_part1')][0]
df = pd.read_csv(os.path.join(DATA_DIR, file))
df.head()


/var/folders/j6/x86rzj9n1s9dkp1dkqyx2h9r0000gn/T/ipykernel_23403/577428276.py:4: DtypeWarning: Columns (1,2,300,312,324) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(DATA_DIR, file))


,Area_code_and_description,Area_code,Area_description,Census_2006_usually_resident_population_count,Census_2013_usually_resident_population_count,Census_2018_usually_resident_population_count,Census_2006_census_night_population_count,Census_2013_census_night_population_count,Census_2018_census_night_population_count,Census_2018_Unit_record_data_source_11_2018_Census_individual_form_CURP,...,Census_2013_Maori_descent_04_Dont_know_CURP,Census_2013_Maori_descent_Total_stated_CURP,Census_2013_Maori_descent_99_Not_elsewhere_included_CURP,Census_2013_Maori_descent_Total_CURP,Census_2018_Maori_descent_01_Māori_descent_CURP,Census_2018_Maori_descent_02_No_Māori_descent_CURP,Census_2018_Maori_descent_04_Dont_know_CURP,Census_2018_Maori_descent_Total_stated_CURP,Census_2018_Maori_descent_99_Not_elsewhere_included_CURP,Census_2018_Maori_descent_Total_CURP
0,SA1 7000000,7000000,7000000,165,144,141,177,150,138,114,...,3,132,12,144,135,6,3,141,0,141
1,SA1 7000001,7000001,7000001,93,105,114,117,108,120,87,...,3,87,15,105,96,18,0,114,0,114
2,SA1 7000002,7000002,7000002,0,0,0,0,0,0,C,...,C,C,C,0,C,C,C,C,C,0
3,SA1 7000003,7000003,7000003,216,171,225,219,168,222,171,...,3,144,24,171,210,15,0,225,0,225
4,SA1 7000004,7000004,7000004,90,102,138,90,105,129,117,...,0,75,27,102,102,30,3,138,0,138
